In [1]:
import pickle
import random
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import datetime

import argparse

parser = argparse.ArgumentParser()

parser.add_argument('--topk', default=1, type=int)
# etc
parser.add_argument('--bptt', default=5, type=int)

parser.add_argument('--data_folder', default='../Data/movielen/1m/', type=str)
parser.add_argument('--data_action', default='item.pickle', type=str)
parser.add_argument('--data_cate', default='cate.pickle', type=str)
parser.add_argument('--data_time', default='time.pickle', type=str)

parser.add_argument("--is_eval", action='store_true')
parser.add_argument('--checkpoint_dir', type=str, default='checkpoint')

parser.add_argument('--valid_start_time', default=0, type=int)
parser.add_argument('--test_start_time', default=0, type=int)

# Get the arguments
args = parser.parse_args([])

In [2]:
# python globalPop_time.py --data_folder ../Data/tmall/100k_unknown_cate/ 
# --data_action item_time.pickle --data_cate cate_time.pickle --data_time time_time.pickle
# --valid_start_time 1512172800 --test_start_time 1512259200 --topk 5


In [3]:
args.data_folder = "../../../Data/tmall/100k_unknown_cate/"
args.data_action = "item_time.pickle"
args.data_cate = "cate_time.pickle"
args.data_time = "time_time.pickle"
args.valid_start_time = 1512172800
args.test_start_time = 1512259200
args.topk = 5

In [4]:
data_action = args.data_folder+args.data_action
data_cate = args.data_folder+args.data_cate
data_time = args.data_folder+args.data_time

action_f = open(data_action, "rb")
action_seq_arr_total = pickle.load(action_f)

cate_f = open(data_cate, "rb")
cate_seq_arr_total = pickle.load(cate_f)

time_f = open(data_time, "rb")
time_seq_arr_totoal = pickle.load(time_f)

action_seq_num = len(action_seq_arr_total)
print("action seq num", action_seq_num)

cate_seq_num = len(cate_seq_arr_total)
print("cate seq num", cate_seq_num)

time_seq_num = len(time_seq_arr_totoal)
print("time seq num", time_seq_num)

valid_start_time = args.valid_start_time
print("valid_start_time", valid_start_time)

test_start_time = args.test_start_time
print("test start time", test_start_time)


action seq num 51275
cate seq num 51275
time seq num 51275
valid_start_time 1512172800
test start time 1512259200


In [5]:
# train_total_action_list = []
# test_total_action_list = []

cate_item_dict = {}
item_cate_dict = {}

# train_total_cate_list = []
# test_total_cate_list = []

# train_total_time_list = []
# test_total_time_list = []

data_list = []

for seq_index in range(action_seq_num):
    action_seq_arr = action_seq_arr_total[seq_index]
    time_seq_arr = time_seq_arr_totoal[seq_index]
    cate_seq_arr = cate_seq_arr_total[seq_index]

    actionNum_seq = len(action_seq_arr)

    valid_start_action_index = -1
    test_start_action_index = -1
        
    for action_index in range(actionNum_seq):  
        item_cur = action_seq_arr[action_index]
        time_cur = time_seq_arr[action_index]
        cate_cur = cate_seq_arr[action_index]
        
        subdata = [seq_index, item_cur, time_cur, cate_cur]
        
        data_list.append(subdata)
        
        if cate_cur not in cate_item_dict:
            cate_item_dict[cate_cur] = []
        if item_cur not in cate_item_dict[cate_cur]:
            cate_item_dict[cate_cur].append(item_cur)
        
        if item_cur not in item_cate_dict:
            item_cate_dict[item_cur] = cate_cur

        if time_cur > valid_start_time:
            if valid_start_action_index == -1:
                valid_start_action_index = action_index

        if time_cur > test_start_time:
            if test_start_action_index == -1:
                test_start_action_index = action_index
                break
    
#     subseq_train = action_seq_arr[:valid_start_action_index]
#     train_total_action_list.append(subseq_train)
    
#     subseq_train_cate = cate_seq_arr[:valid_start_action_index]
#     train_total_cate_list.append(subseq_train_cate)
    
#     subseq_train_time = time_seq_arr[:valid_start_action_index]
#     train_total_time_list.append(subseq_train_time)

#     subseq_test = action_seq_arr[valid_start_action_index: test_start_action_index]
#     test_total_action_list.append(subseq_test)
    
#     subseq_test_cate = cate_seq_arr[valid_start_action_index: test_start_action_index]
#     test_total_cate_list.append(subseq_test_cate)
    
#     subseq_test_time = time_seq_arr[valid_start_action_index: test_start_action_index]
#     test_total_time_list.append(subseq_test_time)

In [6]:
data_df = pd.DataFrame(data_list)
data_df.columns=["userid", "item", "time", "cate"]
print(data_df.head())

   userid   item        time  cate
0       0  38850  1511626494     5
1       0  16692  1511626551     6
2       0  38850  1511626597     5
3       0   7369  1511626761     5
4       0   2395  1511626770    22


In [7]:
data_df.head()

,userid,item,time,cate
0,0,38850,1511626494,5
1,0,16692,1511626551,6
2,0,38850,1511626597,5
3,0,7369,1511626761,5
4,0,2395,1511626770,22


In [8]:
sorted_data_df = data_df.sort_values("time")

In [9]:
sorted_data_df.head()

,userid,item,time,cate
1490175,21790,14282,1511482343,18
2092947,30656,771,1511482459,3
1141659,16740,65018,1511482505,20
1247219,18256,13963,1511483081,51
3156893,46614,67662,1511485308,80


In [10]:
len(sorted_data_df[sorted_data_df.time > valid_start_time])

472732

In [11]:
3467990-472732

2995258

In [12]:
userid_list = sorted_data_df.userid.tolist()
time_list = sorted_data_df.time.tolist()
item_list = sorted_data_df.item.tolist()
cate_list = sorted_data_df.cate.tolist()
action_num = len(cate_list)
print("action_num", action_num)

action_num 3467990


In [13]:
item_num = len(item_cate_dict)
print("item num", item_num)

item num 68007


In [14]:
3050000

3050000

In [16]:
import itertools
train_user_actionlist_dict = {}
recent_user_list = []
recent_usernum = 
recent_actionum = 20

neighbor_topk = 10
action_topk = 5

correct_num = 0
total_num = 0
mrr_num = 0

start_time = datetime.datetime.now()

# for action_index in range(action_num):
for action_index in range(2595258, 3100000):
    if action_index%1e4 == 0:
        print(action_index)
    userid = userid_list[action_index]
    item = item_list[action_index]
    time = time_list[action_index]
    
    if len(recent_user_list) == 0:
        if userid not in train_user_actionlist_dict:
            train_user_actionlist_dict[userid] = []
        train_user_actionlist_dict[userid].append(item)
        recent_user_list.append(userid)
        continue
    
    if time >= valid_start_time:
    
        if userid in train_user_actionlist_dict:

            actionlist_user = train_user_actionlist_dict[userid]
            actionarray_user = np.zeros((1, item_num))
            for action in actionlist_user:
                actionarray_user[0][action] += 1.0

            neighbor_list = recent_user_list[-recent_usernum:]
            neighbor_num = len(neighbor_list)
            actionarray_neighbor = np.zeros((neighbor_num, item_num))
            
            for neighbor_index in range(0, neighbor_num, 5): # 0, ..., recent_usernum-1
                neighbors = [neighbor_list[neighbor_index+i] for i in range(5)]
                
                for action1, action2, action3, action4, action5 in itertools.zip_longest(train_user_actionlist_dict[neighbors[0]], train_user_actionlist_dict[neighbors[1]], train_user_actionlist_dict[neighbors[2]], train_user_actionlist_dict[neighbors[3]], train_user_actionlist_dict[neighbors[4]]):
                    if action1 != None: actionarray_neighbor[neighbor_index][action1] += 1.0
                    if action2 != None: actionarray_neighbor[neighbor_index+1][action2] += 1.0
                    if action3 != None: actionarray_neighbor[neighbor_index+2][action3] += 1.0
                    if action4 != None: actionarray_neighbor[neighbor_index+3][action4] += 1.0
                    if action5 != None: actionarray_neighbor[neighbor_index+4][action5] += 1.0
                    
            user_neighbor_sim = cosine_similarity(actionarray_user, actionarray_neighbor)[0]
            
            neighbor_mask = user_neighbor_sim > 0.1
            neighbor_weight = user_neighbor_sim*neighbor_mask
            neighbor_preds = actionarray_neighbor*(neighbor_weight.reshape(-1, 1))

            neighbor_preds = np.sum(neighbor_preds, axis=0)
            topk_preds = np.argpartition(-neighbor_preds, action_topk)[:action_topk]
            neighbor_vals = neighbor_preds[topk_preds]
            neighbor_vals, topk_preds = zip(*sorted(zip(-neighbor_vals, topk_preds)))

            test_mrr = 0.0
            if item in topk_preds:
                correct_num += 1.0
                test_rank = list(topk_preds).index(item)
                test_mrr = 1.0/(test_rank+1.0)

            total_num += 1.0
            mrr_num += test_mrr
    
    while len(recent_user_list) > recent_usernum:
        recent_user_list.pop(0)
    recent_user_list.append(userid)
    
    if userid not in train_user_actionlist_dict:
        train_user_actionlist_dict[userid] = []
    else:
        while len(train_user_actionlist_dict[userid]) > recent_actionum:
            train_user_actionlist_dict[userid].pop(0)
    train_user_actionlist_dict[userid].append(item)
    
end_time = datetime.datetime.now()
print("duration", end_time-start_time)
print("correct num", correct_num)
print("total num", total_num)
print("recall@{0:d}: {1:.4f}".format(action_topk, correct_num/total_num))
print("mrr@{0:d}: {1:.4f}".format(action_topk, mrr_num/total_num))


2600000
2610000
2620000
2630000
2640000
2650000
2660000
2670000
2680000
2690000
2700000
2710000
2720000
2730000
2740000
2750000
2760000
2770000
2780000
2790000
2800000
2810000
2820000
2830000
2840000
2850000
2860000
2870000
2880000
2890000
2900000
2910000
2920000
2930000
2940000
2950000
2960000
2970000
2980000
2990000
3000000
3010000
3020000
3030000
3040000
3050000
3060000
3070000
3080000
3090000
duration 0:05:25.278220
correct num 174.0
total num 100976.0
recall@5: 0.0017
mrr@5: 0.0011


In [17]:
a = np.array([i*10 for i in range(10)])
random.shuffle(a)
print(a)
# print(np.argpartition(a, 4))
print(a[np.argpartition(a, 4)][:4])

[20 80 70 40 60 50 10 90 30  0]
[10  0 20 30]
